In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.calibration import CalibratedClassifierCV

## Creating historical data csv from 10 tables

- Can ignore and just read in "historical_data_predictive_model.csv"
- Used regular and advanced season stats for the last 5 years

In [9]:
def process_year(year, champion_team):
    
    regular_df = pd.read_csv(f'reg_{year}.csv', header=1)
    regular_df = regular_df.rename(columns={'Unnamed: 1': 'Team'})
    advanced_df = pd.read_csv(f'adv_{year}.csv', header=1)
    advanced_df = advanced_df.rename(columns={'Unnamed: 1': 'Team'})

    data = pd.merge(advanced_df, regular_df, on='Team', suffixes=('_adv', '_reg'))

    data['Stanley_Cup_Champion'] = 0
    data.loc[data['Team'] == champion_team, 'Stanley_Cup_Champion'] = 1
    data['Year'] = year

    return data

In [61]:
years = range(2018, 2023)
champions = ['Washington Capitals*', 'St. Louis Blues*', 'Tampa Bay Lightning*', 'Tampa Bay Lightning*', 'Colorado Avalanche*']
historical_data = pd.concat([process_year(year, champion) for year, champion in zip(years, champions)], ignore_index=True)
historical_data

,Rk_adv,Team,S%_adv,SV%_adv,PDO,CF,CA,CF%,FF,FA,...,SHA,PIM/G,oPIM/G,S,S%_reg,SA,SV%_reg,SO,Stanley_Cup_Champion,Year
0,1,Nashville Predators*,7.6,0.939,101.6,3854,3629,51.5,2857,2689,...,5,11.3,9.6,2641,9.9,2659,0.923,11,0,2018
1,2,Winnipeg Jets*,7.9,0.931,101.0,3815,3590,51.5,2847,2657,...,7,8.5,8.6,2643,10.3,2613,0.917,7,0,2018
2,3,Tampa Bay Lightning*,8.6,0.934,102.0,3879,3640,51.6,2925,2762,...,3,9.2,9.5,2680,10.8,2683,0.913,8,0,2018
3,4,Boston Bruins*,7.3,0.929,100.2,3916,3373,53.7,2948,2532,...,10,9.5,9.6,2703,9.9,2399,0.912,4,0,2018
4,5,Vegas Golden Knights*,7.8,0.927,100.5,3885,3755,50.9,2884,2802,...,5,6.8,7.5,2686,10.0,2521,0.911,5,0,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,28,New Jersey Devils,8.0,0.905,98.6,3801,3734,50.4,2888,2874,...,14,8.1,8.4,2562,9.6,2540,0.881,2,0,2022
152,29,Philadelphia Flyers,7.6,0.919,99.5,3403,3895,46.6,2564,2921,...,11,9.0,9.0,2539,8.3,2785,0.894,1,0,2022
153,30,Seattle Kraken,7.5,0.908,98.3,3508,3415,50.7,2615,2593,...,7,8.5,8.0,2380,8.9,2367,0.880,3,0,2022
154,31,Arizona Coyotes,8.6,0.910,99.7,3118,3977,43.9,2346,3052,...,4,10.2,8.2,2121,9.7,2910,0.894,1,0,2022


In [62]:
historical_data.to_csv('historical_data_predictive_model.csv', index=False)

## Predictive Model

- Selection of random seed affects the prediction differently each time (42 outputs St. Louis)

In [63]:
random_seed = 42

data = historical_data
data = data.drop(columns=['Team'])

X = data.drop(columns=['Stanley_Cup_Champion'])
y = data['Stanley_Cup_Champion']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_seed)

clf = RandomForestClassifier(n_estimators=100, random_state=random_seed)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

data_2022 = data[data['Year'] == 2022]
X_2022 = data_2022.drop(columns=['Stanley_Cup_Champion'])
team_probabilities = clf.predict_proba(X_2022)[:, 1]

top_5_indices = team_probabilities.argsort()[-5:][::-1]
top_5_teams = historical_data.loc[data_2022.index[top_5_indices], 'Team']
top_5_probabilities = team_probabilities[top_5_indices]

for team, prob in zip(top_5_teams, top_5_probabilities):
    print(f"{team}: {prob:.2%}")

predicted_champion = top_5_teams.iloc[0]
print(f"\nPredicted 2022 Stanley Cup Champion: {predicted_champion}")

              precision    recall  f1-score   support

           0       0.97      1.00      0.98        31
           1       0.00      0.00      0.00         1

    accuracy                           0.97        32
   macro avg       0.48      0.50      0.49        32
weighted avg       0.94      0.97      0.95        32

Accuracy: 0.96875
St. Louis Blues*: 16.00%
Nashville Predators*: 12.00%
Colorado Avalanche*: 9.00%
Minnesota Wild*: 8.00%
Tampa Bay Lightning*: 8.00%

Predicted 2022 Stanley Cup Champion: St. Louis Blues*


/Users/lukereath/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/lukereath/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/lukereath/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Averaging multiple random seeds

- When averaging multiple random seeds the correct team is predicted

In [59]:
data = historical_data

teams = data['Team']

data = data.drop(columns=['Team'])

X = data.drop(columns=['Stanley_Cup_Champion'])
y = data['Stanley_Cup_Champion']

random_state_range = range(1, 50)

team_prob_dict = {}

for random_seed in random_state_range:

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_seed)

    clf = RandomForestClassifier(n_estimators=100, random_state=random_seed)
    clf.fit(X_train, y_train)

    data_2022 = data[data['Year'] == 2022]
    X_2022 = data_2022.drop(columns=['Stanley_Cup_Champion'])
    team_probabilities = clf.predict_proba(X_2022)[:, 1] 
    
    for team, prob in zip(teams[data_2022.index], team_probabilities):
        if team not in team_prob_dict:
            team_prob_dict[team] = [prob]
        else:
            team_prob_dict[team].append(prob)

for team in team_prob_dict:
    print(f"{team}: {np.mean(team_prob_dict[team]):.2%}")


Florida Panthers*: 13.08%
Colorado Avalanche*: 51.61%
Carolina Hurricanes*: 5.33%
Toronto Maple Leafs*: 5.59%
Minnesota Wild*: 10.00%
Calgary Flames*: 5.41%
Tampa Bay Lightning*: 7.49%
New York Rangers*: 3.12%
St. Louis Blues*: 11.59%
Boston Bruins*: 4.47%
Edmonton Oilers*: 1.41%
Pittsburgh Penguins*: 2.35%
Washington Capitals*: 0.84%
Los Angeles Kings*: 1.96%
Dallas Stars*: 0.67%
Nashville Predators*: 3.39%
Vegas Golden Knights: 0.92%
Vancouver Canucks: 0.55%
Winnipeg Jets: 1.24%
New York Islanders: 2.10%
Columbus Blue Jackets: 0.80%
San Jose Sharks: 0.02%
Anaheim Ducks: 0.12%
Buffalo Sabres: 0.00%
Detroit Red Wings: 0.02%
Ottawa Senators: 0.33%
Chicago Blackhawks: 0.02%
New Jersey Devils: 0.55%
Philadelphia Flyers: 0.12%
Seattle Kraken: 0.00%
Arizona Coyotes: 1.14%
Montreal Canadiens: 0.53%
